In [7]:
import os
import pandas as pd
import torch
import re
from tqdm import tqdm
from soynlp.normalizer import repeat_normalize
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, BertTokenizer, BertForSequenceClassification
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from textblob import TextBlob

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터셋 로드 및 전처리
file_path = '../외국음식전문점.csv'
df = pd.read_csv(file_path)
df['content'] = df['content'].fillna('')
text_data = df['content'].tolist()

# 전처리 함수 정의
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^가-힣\s]', '', text)
    text = repeat_normalize(text, num_repeats=2)
    stopwords = ['이', '그', '저', '의', '을', '를', '은', '는', '에', '와', '과', '도', '으로', '그리고', '하지만', '그래서']
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text.strip()

preprocessed_data = [preprocess_text(review) for review in text_data]

# 카테고리 및 강화된 키워드 설정
categories = ['맛', '가격', '서비스', '분위기', '추천']
category_keywords = {
    '맛': ['맛', '요리', '음식', '메뉴', '향', '식감', '맛집', '고소', '달콤', '짭짤', '신선', '풍미', '스파이시', '향긋', '촉촉', '바삭', '탱탱', '육즙', '알싸', '상큼', '쫄깃'],
    '가격': ['가격', '비싸', '저렴', '가성비', '비용', '비교', '비율', '가격대', '할인', '구성', '합리적', '저렴하다', '가격 대비', '적당', '경제적', '비용 절감', '가치', '수익', '돈값', '고가', '저가'],
    '서비스': ['서비스', '직원', '응대', '친절', '매너', '대응', '접대', '환대', '서빙', '응대', '고객', '케어', '친절하다', '프로페셔널', '빠르다', '세심하다', '친절도', '고객 관리', '서비스 정신', '상냥', '대접'],
    '분위기': ['분위기', '환경', '인테리어', '공간', '장식', '조명', '음악', '장소', '배경', '아늑하다', '세련', '깔끔', '모던', '클래식', '고급스럽다', '편안하다', '우아', '로맨틱', '분위기 있는', '차분하다', '힐링'],
    '추천': ['추천', '소개', '다음에 또', '재방문', '다시 방문', '친구', '소개하고 싶다', '꼭 오고 싶다', '꼭 다시', '나만 알고 싶다']
}

# 리뷰 카테고리 분류 함수
def categorize_review(review):
    tfidf_vectorizer = TfidfVectorizer()
    keyword_docs = [" ".join(keywords) for keywords in category_keywords.values()]
    tfidf_matrix = tfidf_vectorizer.fit_transform(keyword_docs + [review])
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1]).flatten()
    top_categories = [categories[i] for i in cosine_similarities.argsort()[-2:]]
    return top_categories

# 대표 메뉴 추출
menu_keywords = ["피자", "스파게티", "버거", "스테이크", "초밥"]  # 예시 메뉴 키워드
menu_counter = Counter()
for review in preprocessed_data:
    for menu in menu_keywords:
        if menu in review:
            menu_counter[menu] += 1

top_menus = menu_counter.most_common(3)

# 감정 분석 모델 로드
tokenizer_kobert = BertTokenizer.from_pretrained('monologg/kobert')
model_kobert = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=3).to(device)  # 3개의 레이블: 긍정, 부정, 중립

# 감정 분석 수행 및 카테고리별 비율 분석
sentiment_results = []
category_sentiments = {category: Counter() for category in categories}

for review in tqdm(preprocessed_data, desc="Processing Reviews"):
    inputs_kobert = tokenizer_kobert(review, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        output_kobert = model_kobert(**inputs_kobert)
        sentiment = torch.argmax(output_kobert.logits, dim=1).item()

    categories_found = categorize_review(review)
    sentiment_results.append(sentiment)
    for category in categories_found:
        category_sentiments[category][sentiment] += 1

# 리뷰 요약 모델 로드 및 요약 생성 (간결한 요약 설정)
bart_tokenizer = PreTrainedTokenizerFast.from_pretrained('EbanLee/kobart-summary-v3')
bart_model = BartForConditionalGeneration.from_pretrained('EbanLee/kobart-summary-v3').to(device)

def summarize_reviews(reviews):
    combined_reviews = " ".join(reviews)
    inputs_bart = bart_tokenizer([combined_reviews], return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    summary_ids = bart_model.generate(
        inputs_bart['input_ids'], 
        attention_mask=inputs_bart['attention_mask'], 
        max_length=50,  # 더 짧은 요약
        min_length=20,  
        length_penalty=2.0,  
        num_beams=4, 
        no_repeat_ngram_size=2,  
        repetition_penalty=1.2,  
        early_stopping=True
    )
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # TextBlob을 사용하여 문장 자연스러움 개선
    blob = TextBlob(summary)
    corrected_summary = str(blob.correct())
    
    # 'of' 같은 불필요한 단어 제거
    corrected_summary = re.sub(r'\bof\b', '', corrected_summary)
    
    return corrected_summary

# 가게별 리뷰 종합 요약 및 점수 계산
store_summaries = {}
store_grouped = df.groupby('store_name')['content']

# 순차적으로 가게별 리뷰를 요약 생성 및 점수 계산
for store_name, reviews in tqdm(store_grouped, desc="Processing Stores"):
    try:
        preprocessed_reviews = [preprocess_text(review) for review in reviews]
        summary = summarize_reviews(preprocessed_reviews)

        # 감성 비율 계산
        sentiments = [sentiment_results[i] for i, r in enumerate(preprocessed_data) if r in reviews]
        sentiment_counts = Counter(sentiments)
        total_reviews = len(sentiments)
        sentiment_ratios = {label: sentiment_counts.get(i, 0) / total_reviews for i, label in enumerate(['Negative', 'Neutral', 'Positive'])}

        # 카테고리별 점수 계산
        category_scores = {}
        for category in categories:
            category_count = sum(category_sentiments[category].values())
            category_scores[category] = category_sentiments[category][2] / category_count if category_count > 0 else 0  # 긍정 점수

        # 최종 점수 계산 (예: 감성 비율과 카테고리 점수의 평균)
        overall_score = (sentiment_ratios['Positive'] + sum(category_scores.values()) / len(categories)) / 2

        # 결과 저장
        store_summaries[store_name] = {
            'Summary': summary,
            'Sentiment Ratios': sentiment_ratios,
            'Category Scores': category_scores,
            'Overall Score': overall_score
        }

    except Exception as e:
        print(f"Error processing store {store_name}: {e}")

# 현재 날짜와 시간을 가져와서 파일 이름에 포함
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
file_name = f'{current_time}_store_review_summaries.csv'


# 결과를 CSV 파일로 저장
final_results = pd.DataFrame.from_dict(store_summaries, orient='index')
final_results.to_csv(file_name)

print(f"Results saved to {file_name}")

# 시각화 및 이미지 저장
for store_name, data in store_summaries.items():
    # 감성 비율 시각화
    sentiment_ratios = data['Sentiment Ratios']
    plt.figure(figsize=(8, 4))
    sns.barplot(x=list(sentiment_ratios.keys()), y=list(sentiment_ratios.values()))
    plt.title(f'{store_name} - Sentiment Distribution')
    sentiment_image_file = f'{current_time}_{store_name}_sentiment_distribution.png'
    plt.savefig(sentiment_image_file)
    plt.close()

    # 카테고리 점수 시각화
    category_scores = data['Category Scores']
    plt.figure(figsize=(8, 4))
    sns.barplot(x=list(category_scores.keys()), y=list(category_scores.values()))
    plt.title(f'{store_name} - Category Scores')
    category_image_file = f'{current_time}_{store_name}_category_scores.png'
    plt.savefig(category_image_file)
    plt.close()

print("All processing and saving of results completed.")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
C:\Users\G-01\AppData\Roaming\Python\Python38\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Processing

Error processing store (주)애브릿지: division by zero


Processing Stores:   0%|          | 2/737 [00:08<54:10,  4.42s/it]

Error processing store (주)플레이버타운: division by zero


Processing Stores:   0%|          | 3/737 [00:13<52:50,  4.32s/it]

...

Error processing store 흑석호치민: division by zero


Processing Stores: 100%|█████████▉| 736/737 [52:17<00:04,  4.06s/it]

Error processing store 히타나: division by zero


Processing Stores: 100%|██████████| 737/737 [52:22<00:00,  4.26s/it]

Error processing store 힛힛: division by zero
Results saved to 20240831_124204_store_review_summaries.csv
All processing and saving of results completed.


Results saved to 20240831_104000_store_review_summaries.csv
All processing and saving of results completed.
